<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Базовые-модели" data-toc-modified-id="Базовые-модели-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Базовые модели</a></span></li><li><span><a href="#Bert-from-paper" data-toc-modified-id="Bert-from-paper-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Bert from paper</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» c  BERT



Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [23]:
# !pip install torch torchmetrics transformers

In [24]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from tqdm.notebook import tqdm


from tqdm import notebook 

import torch
from transformers import AutoTokenizer, BertModel
import transformers

import platform

# from google.colab import drive
# drive.mount('/content/drive')

PATH = Path('drive/MyDrive/Colab_Notebooks/')

In [25]:

def evaluate_and_report(report_df:pd.DataFrame, 
                        model, X, y, 
                        model_name, scoring='f1', 
                        predict_proba=False, 
                        thresh=0.3):
    '''
    Function to evaluate model and make report Dataframe
    '''
    
    report = pd.DataFrame()
    report.index = [model_name]
    if predict_proba:
        predicted = (model.predict_proba(X)[:,1] >= thresh).astype(int)
        report['f1'] = f1_score(y, predicted)
                       
                       
    else:
        predicted = model.predict(X)
        cv_marks = cross_val_score(model, X, y, scoring=scoring, cv=5, n_jobs=-1)
        print(f"cross_vall_score: {cv_marks}")
        report['f1'] = cv_marks.mean()
    
    
    
    report.loc[:,'precision_0'] = precision_score(
        y, predicted, pos_label=0)
    report.loc[:,'precision_1'] = precision_score(
        y, predicted, pos_label=1)
    report.loc[:,'recall_0'] = recall_score(
        y, predicted, pos_label=0)
    report.loc[:,'recall_1'] = recall_score(
        y, predicted, pos_label=1)
    
    
    report_df = report_df.append(report)
    return report_df

def choose_platform(platform):
    if platform == 'Darwin':  # for macos m1 gpu acceleration
        device = 'mps'
    elif platform == 'Linux' and torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'
    return device

## Подготовка

In [26]:
local_path = '../datasets/toxic_comments.csv'
cloud_path = '/datasets/toxic_comments.csv'
colab_path = Path(PATH / 'datasets/toxic_comments.csv')

if os.path.exists(local_path):
    df = pd.read_csv(local_path)
elif os.path.exists(cloud_path):
    df = pd.read_csv(cloud_path)
elif os.path.exists(colab_path):
    df = pd.read_csv(colab_path)
else:
    print('something wrong! Check path')

In [27]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Базовые модели

### Bert from paper

In [28]:
# original paper text:
# https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [29]:
DATA_SIZE = int(len(df) * .05) # num of objects hand over pytorch. if whole dataset - len(df)
MODEL_NAME = 'bert-base-cased' # parameter for download pretrained model and tokenizer
MAX_LENGTH = 400
BATCH_SIZE = 2
LEARNING_RATE = 3e-5 

In [30]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) #AutoTokenizer.from_pretrained('bert-base-uncased')

In [31]:
df = df.sample(DATA_SIZE)
sentences = df.text.values
labels = df.toxic.values

In [32]:
# select random sentence
rand_idx = np.random.randint(df.shape[0])

# Print the original sentence.
print(' Original: ', sentences[rand_idx])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[rand_idx]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[rand_idx])))

 Original:  Wykagyl is the area / community which consists of many neighborhoods including Bonnie Crest, Wykagyl Estates, Victory Park, Bon Air, Forest Knolls etc. The information listed on google maps, while helpful, is not the most reliable source of information or the most accurate.
Tokenized:  ['w', '##yk', '##ag', '##yl', 'is', 'the', 'area', '/', 'community', 'which', 'consists', 'of', 'many', 'neighborhoods', 'including', 'bonnie', 'crest', ',', 'w', '##yk', '##ag', '##yl', 'estates', ',', 'victory', 'park', ',', 'bon', 'air', ',', 'forest', 'kn', '##oll', '##s', 'etc', '.', 'the', 'information', 'listed', 'on', 'google', 'maps', ',', 'while', 'helpful', ',', 'is', 'not', 'the', 'most', 'reliable', 'source', 'of', 'information', 'or', 'the', 'most', 'accurate', '.']
Token IDs:  [1059, 15922, 8490, 8516, 2003, 1996, 2181, 1013, 2451, 2029, 3774, 1997, 2116, 11681, 2164, 12220, 11146, 1010, 1059, 15922, 8490, 8516, 8707, 1010, 3377, 2380, 1010, 14753, 2250, 1010, 3224, 14161, 1451

In [33]:
# Tokenize all of the sentences and map the tokens to thier word IDs.


input_ids = []
attention_masks = []


# For every sentence...
for i, sent in enumerate(notebook.tqdm(sentences)):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length=64,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])


  0%|          | 0/7978 [00:00<?, ?it/s]

Original:  Geez, are you forgetful!  We've already discussed why Marx  was  not an anarchist, i.e. he wanted to use a State to mold his 'socialist man.'  Ergo, he is a statist - the opposite of an  anarchist.  I know a guy who says that, when he gets old and his teeth fall out, he'll quit eating meat.  Would you call him a vegetarian?
Token IDs: tensor([  101, 20277,  2480,  1010,  2024,  2017,  5293,  3993,   999,  2057,
         1005,  2310,  2525,  6936,  2339, 13518,  2001,  2025,  2019, 18448,
         1010,  1045,  1012,  1041,  1012,  2002,  2359,  2000,  2224,  1037,
         2110,  2000, 18282,  2010,  1005,  6102,  2158,  1012,  1005,  9413,
         3995,  1010,  2002,  2003,  1037, 28093,  2923,  1011,  1996,  4500,
         1997,  2019, 18448,  1012,  1045,  2113,  1037,  3124,  2040,  2758,
         2008,  1010,  2043,   102])


In [34]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 75-25 train-validation split.

# Calculate the number of samples to include in each set.

train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
test_size = val_size // 2


# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


val_dataset, test_dataset = random_split(val_dataset, [test_size,
                                                       (test_size + 1 if val_size % 2 else test_size)]) # check is even number

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


5,983 training samples
1,995 validation samples


In [35]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
# batch_size = 32 

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
        )
test_dataloader = DataLoader(
    test_dataset,
    sampler = SequentialSampler(test_dataset),
    batch_size = BATCH_SIZE
)

In [36]:
from transformers import BertForSequenceClassification, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
# model.cuda()
device = choose_platform(platform.system()) #'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

mps


In [37]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
#     !nvidia-smi
print(f'!!! TRAINING ON  {device}')

!!! TRAINING ON  mps


In [38]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [39]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
from torch.optim import AdamW
optimizer = AdamW(model.parameters(),
                  lr = LEARNING_RATE, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [40]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [41]:
import numpy as np
# from torchmetrics.classification import BinaryF1Score

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy_f1(preds, labels):
    metrics_dict = {}
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    # print(type(preds))
    # print(preds)
    f1 = f1_score(
        torch.from_numpy(preds), 
        torch.from_numpy(labels)
        )
    metrics_dict['accuracy'] =  np.sum(pred_flat == labels_flat) / len(labels_flat)
    metrics_dict['f1'] = f1
    # return np.sum(pred_flat == labels_flat) / len(labels_flat)
    return metrics_dict

In [42]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [43]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in tqdm(range(0, epochs)):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(tqdm(train_dataloader)):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels,
                            return_dict=False)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
#         print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_f1 = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for i, batch in enumerate(tqdm(validation_dataloader)):
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels,
                                  return_dict=False)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        metrics_dict = flat_accuracy_f1(logits, label_ids)
        total_eval_accuracy += metrics_dict['accuracy']
        total_eval_f1 += metrics_dict['f1']
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    print(f"   F1-score: {avg_val_f1:.2f}")

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

  0%|          | 0/2 [00:00<?, ?it/s]


======== Epoch 1 / 2 ========
Training...


  0%|          | 0/2992 [00:00<?, ?it/s]

  Batch    40  of  2,992.    Elapsed: 0:00:08.
  Batch    80  of  2,992.    Elapsed: 0:00:15.
  Batch   120  of  2,992.    Elapsed: 0:00:22.
  Batch   160  of  2,992.    Elapsed: 0:00:29.
  Batch   200  of  2,992.    Elapsed: 0:00:35.
  Batch   240  of  2,992.    Elapsed: 0:00:43.
  Batch   280  of  2,992.    Elapsed: 0:00:50.
  Batch   320  of  2,992.    Elapsed: 0:00:57.
  Batch   360  of  2,992.    Elapsed: 0:01:04.
  Batch   400  of  2,992.    Elapsed: 0:01:11.
  Batch   440  of  2,992.    Elapsed: 0:01:18.
  Batch   480  of  2,992.    Elapsed: 0:01:25.
  Batch   520  of  2,992.    Elapsed: 0:01:32.
  Batch   560  of  2,992.    Elapsed: 0:01:39.
  Batch   600  of  2,992.    Elapsed: 0:01:46.
  Batch   640  of  2,992.    Elapsed: 0:01:52.
  Batch   680  of  2,992.    Elapsed: 0:01:59.
  Batch   720  of  2,992.    Elapsed: 0:02:06.
  Batch   760  of  2,992.    Elapsed: 0:02:13.
  Batch   800  of  2,992.    Elapsed: 0:02:20.
  Batch   840  of  2,992.    Elapsed: 0:02:27.
  Batch   880

  0%|          | 0/499 [00:00<?, ?it/s]

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [25]:
torch.cuda.empty_cache()

## Выводы